In [ ]:
pip install espn_api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for espn-api: filename=espn_api-0.26.2-py3-none-any.whl size=55715 sha256=7a063178cedfdacfa72f27a9300533a0e052a85d2224523cd33a66857bfcac0f
  Stored in directory: /root/.cache/pip/wheels/ba/55/7e/aceac40e33766956afee97b29a15682cd4faadf55edf0b262a
Successfully built espn-api


In [ ]:
# Import Basketball API and all relevant libraries
from espn_api.basketball import League
from espn_api.basketball import Player

import pandas as pd
import numpy as np
import math

import time
import datetime as dt
from dateutil import tz

In [ ]:
# Setup the league variables for this year and previous year.
leaguePreviousYear = League(league_id=1483720569, year=2022, espn_s2='AEAcEt%2BMhB7Fge2Z8ytHMzkMCzI3Pv9P2iScn2FG4%2BjItbmIma%2BKoAP32crD2jneK0ZSIpkCeuMewE2tWpBSh8V427kylczAMaNqUpWA3XlpILa8IUyjLY%2BiKLeNdEobg0TBvOfYzWIxEJYdE5%2FctLuNNvV9GVpzc4u2I6FTdEleJgkb4FLU1NGuK8CAWppO8eI0cYgIWpWfNfFHIw9NmlqFTH57vdXgYRGNq3pGJuLZxgVIq54ghOwrsynfR8uQ2yjhAwI7n74p5eqgMJ9Ifnh74mvMGOFeArbTx9KMKkYeWg%3D%3D', 
                swid='{AA6619A1-D92E-4220-BECC-9A41FA2A0B0D}')
league = League(league_id=1483720569, year=2023, espn_s2='AEAcEt%2BMhB7Fge2Z8ytHMzkMCzI3Pv9P2iScn2FG4%2BjItbmIma%2BKoAP32crD2jneK0ZSIpkCeuMewE2tWpBSh8V427kylczAMaNqUpWA3XlpILa8IUyjLY%2BiKLeNdEobg0TBvOfYzWIxEJYdE5%2FctLuNNvV9GVpzc4u2I6FTdEleJgkb4FLU1NGuK8CAWppO8eI0cYgIWpWfNfFHIw9NmlqFTH57vdXgYRGNq3pGJuLZxgVIq54ghOwrsynfR8uQ2yjhAwI7n74p5eqgMJ9Ifnh74mvMGOFeArbTx9KMKkYeWg%3D%3D', 
                swid='{AA6619A1-D92E-4220-BECC-9A41FA2A0B0D}')

## Constants

In [ ]:
#Constants
dayZero = dt.datetime(2022,10,16).timestamp()
dayZerodt = dt.datetime(2022,10,16)

## Getter Functions

In [ ]:
#Base level outside of API/ helper functions

def getCurrentWeekNum():
  today = dt.datetime.today().timestamp()
  weeks = (today - dayZero) / 604800
  weekNum = math.ceil(weeks)
  return weekNum

def getLastWeekNum():
  return getCurrentWeekNum() - 1

def createPlayerDataFrame(player):
  playerStats = pd.DataFrame(player.stats).transpose()
  playerStatsIndex = playerStats.index
  playerStats['name'] = player.name
  playerStats = playerStats.set_index(['name',playerStatsIndex])
  return playerStats

#Base level of API, but return all functions as dataframes for easier manipulation. 
#The functions will return API objects, and so they will need to be transformed appropriately in whatever function they are later used in.

#Get all teams not in ranked order
def getTeams():
  return pd.DataFrame(league.teams)

#Get standings
def getStandings():
  return pd.DataFrame(league.standings())

#Get Recent Activity for activity
def getRecentActivity(size, activity):
  return pd.DataFrame(league.recent_activity(size=size,msg_type=activity))

#Get all free agents
def getFreeAgents():
  return pd.DataFrame(league.free_agents(size=1000))

#Get scoreboard for weekNum week
def getScoreboard(weekNum):
  return pd.DataFrame(league.scoreboard(weekNum))

#Get all boxscores for weekNum week
def getBoxScores(weekNum):
  return pd.DataFrame(league.box_scores(weekNum))

#Get all boxscores for weekNum week
def getBoxScores_adv(scoring_period, matchup_total):
  return pd.DataFrame(league.box_scores(scoring_period=scoring_period, matchup_total=matchup_total))

#Retrieve saved standings from last week.
def getSavedStandings():
  weekNum = getCurrentWeekNum()
  with open('/content/drive/MyDrive/Personal Projects/NBA_Fantasy/Standings - Week of {0}.txt'.format(dayZero + dt.timedelta(weeks=weekNum-1)), 'r') as f:
    standings = f.read().splitlines()
    standings = pd.DataFrame(standings)
    standings.index += 1
    standings.rename(columns={0:"Team"}, inplace=True)
    standings['Rank'] = standings.index
  return standings

# Advanced getter functions, using and transforming base getter functions.

#Get all players on rosters
def getRosterPlayers():
  teams = getTeams()
  players = teams.apply(lambda team: team[0].roster, axis=1)
  players = players.explode()
  players = players.reset_index(drop=True)
  return players

#Get all players in the league. This will be all players on rosters + all free agents.
def getPlayers():
  return pd.concat([getRosterPlayers(), getFreeAgents()])

#Get a multi-indexed dataframe that contains the stats of all players
def getPlayerStats():
  players = getPlayers()
  playerStats = []
  for row in players.itertuples():
    playerStats.append(createPlayerDataFrame(row._1))
  playerStats = pd.concat(playerStats)
  return playerStats

#Get total fantasy points scored
def getTotalFPTS(weekNum):
  allScoreboards = []
  for week in range(weekNum):
    scoreboard = getScoreboard(week)
    scoreboard['AwayTeam'] = scoreboard.apply(lambda matchup: matchup[0].away_team, axis=1)
    scoreboard['AwayScore'] = scoreboard.apply(lambda matchup: matchup[0].away_final_score, axis=1) 
    scoreboard['HomeTeam'] = scoreboard.apply(lambda matchup: matchup[0].home_team, axis=1)
    scoreboard['HomeScore'] = scoreboard.apply(lambda matchup: matchup[0].home_final_score, axis=1)
    scoreboard_away = scoreboard[['AwayTeam', 'AwayScore']].rename(columns={'AwayTeam':'Team', 'AwayScore':week})
    scoreboard_home = scoreboard[['HomeTeam', 'HomeScore']].rename(columns={'HomeTeam':'Team', 'HomeScore':week})

    Scoreboard = pd.concat([scoreboard_away, scoreboard_home], axis=0).set_index('Team', drop=True)

    allScoreboards.append(Scoreboard)

  totalFPTS = pd.concat(allScoreboards, axis=1)
  totalFPTS['Total'] = totalFPTS.apply(lambda x: sum(x), axis=1)
  totalFPTS.index = totalFPTS.index.map(lambda team: team.team_name)
  totalFPTS = totalFPTS.reset_index() 

  return totalFPTS

def getComebackWins():
  lastWeekNum = getLastWeekNum() 
  startPeriod = (lastWeekNum * 7) - 6
  endPeriod = lastWeekNum * 7 + 1
  allBoxScores = []
  for period in range(startPeriod, endPeriod):
    boxscore = getBoxScores_adv(scoring_period=period, matchup_total=False)
    boxscore['AwayTeam'] = boxscore.apply(lambda matchup: matchup[0].away_team, axis=1)
    boxscore['AwayScore'] = boxscore.apply(lambda matchup: matchup[0].away_score, axis=1) 
    boxscore['HomeTeam'] = boxscore.apply(lambda matchup: matchup[0].home_team, axis=1)
    boxscore['HomeScore'] = boxscore.apply(lambda matchup: matchup[0].home_score, axis=1)
    boxscore_away = boxscore[['AwayTeam', 'AwayScore']].rename(columns={'AwayTeam':'Team', 'AwayScore':period-startPeriod+1})
    boxscore_home = boxscore[['HomeTeam', 'HomeScore']].rename(columns={'HomeTeam':'Team', 'HomeScore':period-startPeriod+1})
    boxscores = pd.concat([boxscore_away, boxscore_home], axis=0).set_index('Team', drop=True)

    allBoxScores.append(boxscores)

  totalscore = pd.concat(allBoxScores, axis=1)
  totalscore['Total'] = totalscore.apply(lambda x: sum(x), axis=1)
  totalscore['Last Day'] = totalscore.apply(lambda x: sum(x[0:6]),axis=1)
  totalscore.index = totalscore.index.map(lambda team: team.team_name)

  #Reconstruct matchups and compare winner to last day scores
  matchupInfo = getBoxScores(lastWeekNum)
  matchupInfo['AwayTeam'] = matchupInfo.apply(lambda matchup: matchup[0].away_team.team_name, axis=1)
  matchupInfo['AwayFinalScore'] = matchupInfo.apply(lambda matchup: matchup[0].away_score, axis=1)
  matchupInfo['HomeTeam'] = matchupInfo.apply(lambda matchup: matchup[0].home_team.team_name, axis=1)
  matchupInfo['HomeFinalScore'] = matchupInfo.apply(lambda matchup: matchup[0].home_score, axis=1)
  matchupInfo['AwayLastScore'] = matchupInfo.apply(lambda row: totalscore['Last Day'].loc[row['AwayTeam']], axis=1)
  matchupInfo['HomeLastScore'] = matchupInfo.apply(lambda row: totalscore['Last Day'].loc[row['HomeTeam']], axis=1)
  matchupInfo['WINNER'] = matchupInfo.apply(lambda matchup: matchup[0].home_team.team_name if matchup[0].winner=='HOME' else matchup[0].away_team.team_name, axis=1)
  matchupInfo = matchupInfo[['AwayTeam', 'AwayFinalScore', 'AwayLastScore', 'HomeTeam', 'HomeFinalScore', 'HomeLastScore', 'WINNER']]

  comebackWins = matchupInfo.query('(AwayFinalScore > HomeFinalScore & AwayLastScore < HomeLastScore) | (HomeFinalScore > AwayFinalScore & HomeLastScore < AwayLastScore)')

  return comebackWins

## Setter Functions

In [ ]:
#Save current standings
def saveStandings():
  standings = getStandings()
  weekNum = getCurrentWeekNum()
  with open('/content/drive/MyDrive/Personal Projects/NBA_Fantasy/Standings - Week of {0}.txt'.format(dayZerodt + dt.timedelta(weeks=weekNum)), 'w') as f:
    for row in standings.itertuples():
      f.write(row[1].team_name)
      f.write('\n')

In [ ]:
saveStandings()

In [ ]:
getTotalFPTS(4)

,Team,0,1,2,3,Total
0,Big Gabes,378.0,1186.0,1522.0,1582.0,4668.0
1,Powie wowie Boulette,303.0,1124.0,1366.0,1445.0,4238.0
2,Team Big tings eh,272.0,1286.0,1343.0,981.0,3882.0
3,Chris’s Team Ded Inside,305.0,1117.0,1516.0,1149.0,4087.0
4,Blubber Conquest,422.0,1302.0,1400.0,1573.0,4697.0
5,Your Mom luvs Kareempie,298.0,1078.0,1631.0,1232.0,4239.0
6,House Yarish,377.0,1133.0,1319.0,1185.0,4014.0
7,House Huynh,335.0,1303.0,1343.0,1050.0,4031.0
8,Grand blue Straw hats,334.0,1026.0,1158.0,1228.0,3746.0
9,brooklyn park splashers,341.0,1162.0,1423.0,1042.0,3968.0


## OLD SCRIPTS BELOW


In [ ]:
## FINISHED
#Standings
#Last week's closest matchup
#Best performing player of the week
#Best performing player not on a team
#Upcoming power matchups
#Power matchup results from last week

## NOT FINISHED
#Best Free agent pickup

In [ ]:
# Helper functions

#First day of NBA fantasy on ESPN begins Oct 17, 2022 (This is scoring_period=1). The first sunday (scoring_period=0) is Oct 16, 2022.
firstSunday = dt.datetime(2022,10,16).timestamp()

#Get previous week number
def getCurrentWeekNum():
  firstSunday = dt.datetime(2022,10,16).timestamp()
  today = dt.datetime.today().timestamp()
  weeks = (today - firstSunday) / 604800
  weekNum = math.ceil(weeks)
  return weekNum

def getLastWeekNum():
  return getCurrentWeekNum() - 1

#Get scoring periods
def getScoringPeriods(startDate):
  startDate = (startDate / 1000) - 5*3600
  firstSunday = dt.datetime(2022,10,16).timestamp()
  #today = dt.datetime.today().timestamp()
  #endPeriod = ((today - firstSunday) - 2*86400) / 86400
  endPeriod = (dt.timedelta(weeks=getLastWeekNum())).total_seconds() / 86400
  endPeriod = round(endPeriod) 
  startPeriod = (startDate - firstSunday) / 86400
  startPeriod = round(startPeriod)

  return startPeriod, endPeriod


In [ ]:
#Save current standings
def saveStandings():
  with open('/content/drive/MyDrive/Personal Projects/NBA_Fantasy/Standings - Week of {0}.txt'.format(dt.date.today()), 'w') as f:
    for team in league.standings():
      f.write(team.team_name)
      f.write('\n')

#Retrieve saved standings from last week.
def getSavedStandings():
  with open('/content/drive/MyDrive/Personal Projects/NBA_Fantasy/Standings - Week of {0}.txt'.format(dt.date.today()-dt.timedelta(days=7)), 'r') as f:
    standings = f.read().splitlines()
    standings = pd.DataFrame(standings)
    standings.index += 1
    standings.rename(columns={0:"Team"}, inplace=True)
    standings['Rank'] = standings.index
  return standings

In [ ]:
#Standings
#MONDAY MORNING
def getStandings():
  standings = pd.DataFrame(league.standings())
  standings.index += 1
  standings.rename(columns={0:"Team"}, inplace=True)
  standings['Team'] = standings.apply(lambda x: x['Team'].team_name, axis=1)
  standings['Rank'] = standings.index
  return standings

#Last week's closest matchup
#MONDAY MORNING

def getClosestMatchup():
  weekNum = getLastWeekNum()
  matchups = league.box_scores(weekNum)

  scoreDifferences = list(map(lambda x: abs(x.home_score - x.away_score), matchups))
  matchupScores = zip(matchups, scoreDifferences)
  closeMatchupScores = sorted(matchupScores, key = lambda x: x[1])

  closestMatchups = [matchup for matchup in closeMatchupScores if abs(matchup[1] - closeMatchupScores[0][1]) < 30]

  closestMatchupsdf = pd.DataFrame(closestMatchups)
  closestMatchupsdf['AWAY'] = closestMatchupsdf.apply(lambda x: x[0].away_team.team_name, axis=1)
  closestMatchupsdf['HOME'] = closestMatchupsdf.apply(lambda x: x[0].home_team.team_name, axis=1) 
  closestMatchupsdf['WINNER'] = closestMatchupsdf.apply(lambda x: x[0].away_team.team_name if x[0].winner == "AWAY" else x[0].home_team.team_name, axis=1)
  closestMatchupsdf = closestMatchupsdf.rename(columns = {0: "Boxscore Object", 1: "Difference"})

  return closestMatchupsdf

#Best perfoming player of the week on a team
#MONDAY MORNING

def getBestPlayersInLineups():
  teams = league.teams
  playersOnRosters = [(player, team.team_name) for team in teams for player in team.roster]

  bestPlayersInLineups = [(player[0].stats['2023_last_7']['applied_total'], player[0].name, player[1]) for player in playersOnRosters]
  bestPlayersInLineups = sorted(bestPlayersInLineups, key = lambda x: x[0], reverse=True)
  bestPlayersInLineups = bestPlayersInLineups[:5]

  bestPlayersdf = pd.DataFrame(bestPlayersInLineups)
  bestPlayersdf = bestPlayersdf.rename(columns={0:"Total Score", 1:"Player", 2:"Team"})

  return bestPlayersdf

#Best performing player not on a team
#MONDAY MORNING
def getBestFreeAgents():
  freeAgents = league.free_agents(size=500)

  bestFreeAgents = [(fa.stats['2023_last_7']['applied_total'], fa.name) for fa in freeAgents if (fa.projected_avg_points) != 0]
  bestFreeAgents = sorted(bestFreeAgents, key = lambda x: x[0], reverse=True)
  bestFreeAgents = bestFreeAgents[:5]

  bestFreeAgentsdf = pd.DataFrame(bestFreeAgents)
  bestFreeAgentsdf = bestFreeAgentsdf.rename(columns={0:"Total Score", 1:"Player"})

  return bestFreeAgentsdf

#Upcoming power matchups and previous power matchup results
#MONDAY MORNING

def rankSum(standings, matchup):
  sum = standings[standings['Team']==matchup.home_team.team_name]['Rank'].iloc[0] + standings[standings['Team']==matchup.away_team.team_name]['Rank'].iloc[0]
  return sum

def getPowerMatchups(prev):
  if prev == True:
    standings = getSavedStandings()
    weekNum = getLastWeekNum()
  else:
    standings = getStandings()  
    weekNum = getCurrentWeekNum()
  matchups = league.box_scores(weekNum)

  rankSums = list(map(lambda x: rankSum(standings, x), matchups))
  rankSums = list(enumerate(rankSums))
  rankSums = sorted(rankSums, key = lambda x: x[1])
  bestMatchups = [rank for rank in rankSums if rank[1]==rankSums[0][1]]
  bestMatchups = [matchups[matchup[0]] for matchup in bestMatchups]

  bestMatchupsdf = pd.DataFrame(bestMatchups)
  bestMatchupsdf['AWAY'] = bestMatchupsdf.apply(lambda x: x[0].away_team.team_name, axis=1)
  bestMatchupsdf['AWAY RANK'] = bestMatchupsdf.apply(lambda x: standings[standings['Team']==x[0].away_team.team_name]['Rank'].iloc[0], axis=1)
  bestMatchupsdf['HOME'] = bestMatchupsdf.apply(lambda x: x[0].home_team.team_name, axis=1)
  bestMatchupsdf['HOME RANK'] = bestMatchupsdf.apply(lambda x: standings[standings['Team']==x[0].home_team.team_name]['Rank'].iloc[0], axis=1) 

  return bestMatchupsdf

In [ ]:
#Best free agent pickup
#league.recent_activity()[0].actions[0][1]
def getLineups():
  teams = league.teams
  lineup = [player for team in teams for player in team.roster]
  return lineup

def getPickups():
  activities = league.recent_activity(msg_type = 'FA')
  pickups = [(activity.actions[0][2], activity.actions[0][0], activity.date) for activity in activities]
  return pickups

def pickupsStillOnTeams():
  pickups = getPickups()
  lineups = getLineups()
  pickupsOnTeams = [pickup for player in lineups for pickup in pickups if pickup[0]==player.name]
  return pickupsOnTeams

#date = dt.datetime.fromtimestamp(pickups[0][2] / 1000, dt.timezone(dt.timedelta(hours=-5)))
def getPickupScores():
  pickups = pickupsStillOnTeams()
  pickupsTotalScore = []
  for pickup in pickups:
    pickup = list(pickup)
    pickup.append(getScoringPeriods(pickup[2]))
    totalScore = 0
    gamesPlayed = 0
    scoringPeriodStart = pickup[3][0]
    scoringPeriodEnd = pickup[3][1]
    while(scoringPeriodEnd >= scoringPeriodStart):
      boxscores = league.box_scores(scoring_period=scoringPeriodStart, matchup_total=False)
      for boxscore in boxscores:
        if boxscore.away_team == pickup[1]:
          for player in boxscore.away_lineup:
            if (player.name == pickup[0]) == True:
              totalScore += player.points
              if player.points != 0:
                gamesPlayed += 1
        elif (boxscore.home_team == pickup[1]) == True:
          for player in boxscore.home_lineup:
            if player.name == pickup[0]:
              totalScore += player.points
              if player.points != 0:
                gamesPlayed += 1
      scoringPeriodStart += 1
    pickup.append(totalScore)
    if gamesPlayed != 0:
      pickup.append(totalScore / gamesPlayed)
    else:
      pickup.append(0)
    pickupsTotalScore.append(pickup)
  pickupsTotalScoredf = pd.DataFrame(pickupsTotalScore)
  pickupsTotalScoredf = pickupsTotalScoredf.rename(columns = {0:"Player", 1:"Team", 2:"Pickup Date (Epoch Seconds UTC)", 3:"Scoring Period Range", 4:"Total Points", 5:"Avg Points"})
  pickupsTotalScoredf['period diff'] = pickupsTotalScoredf.apply(lambda x: x['Scoring Period Range'][1] - x['Scoring Period Range'][0], axis=1)
  pickupsTotalScoredf = pickupsTotalScoredf[pickupsTotalScoredf['period diff'] >= 0]
  pickupsTotalScoredf['Team'] = pickupsTotalScoredf.apply(lambda x: x['Team'].team_name, axis=1)
  #pickupsTotalScoredf = pickupsTotalScoredf.drop()
  return pickupsTotalScoredf

In [ ]:
#New Functions to add:
#Underdog victory: matchup with large rank difference where lower ranked team won
#Win Streak: Display teams that are on a 2+ win streak
#Hospital Team Win: Teams that won with 2 or more players injured.

In [ ]:
#THESE FUNCTIONS DO NOT WORK IN POWER BI
def getPrevPowerMatchupResults():
  prevStandings = getSavedStandings()
  prevPowerMatchups = getPowerMatchups(True)
  prevPowerMatchups['WINNER'] = prevPowerMatchups.apply(lambda x: x[0].home_team.team_name if x[0].winner == 'HOME' else x[0].away_team.team_name, axis=1)

  return prevPowerMatchups

#Underdog victory:
def getUnderdogWins():
  standings = getSavedStandings()
  matchups = league.box_scores(getLastWeekNum())

  rankSums = list(map(lambda x: rankSum(standings, x), matchups))
  rankSums = list(enumerate(rankSums))
  rankSums = sorted(rankSums, key = lambda x: x[1], reverse=True)

  worstMatchups = rankSums[:2]
  worstMatchups = [(matchups[matchup[0]], (standings[standings['Team']==matchups[matchup[0]].away_team.team_name]['Rank'].iloc[0],standings[standings['Team']==matchups[matchup[0]].home_team.team_name]['Rank'].iloc[0])) for matchup in worstMatchups]

  underdogWins = []

  for matchup in worstMatchups:
    diff = matchup[1][1] - matchup[1][0]
    if diff > 0 and matchup[0].winner == 'HOME':
      underdogWins.append(matchup)
    if diff < 0 and matchup[0].winner == 'AWAY':
      underdogWins.append(matchup)

  underdogWins = pd.DataFrame(underdogWins)
  underdogWins['AWAY'] = underdogWins.apply(lambda x: x[0].away_team.team_name, axis=1)
  underdogWins['AWAY RANK'] = underdogWins.apply(lambda x: standings[standings['Team']==x[0].away_team.team_name]['Rank'].iloc[0], axis=1)
  underdogWins['HOME'] = underdogWins.apply(lambda x: x[0].home_team.team_name, axis=1)
  underdogWins['HOME RANK'] = underdogWins.apply(lambda x: standings[standings['Team']==x[0].home_team.team_name]['Rank'].iloc[0], axis=1)
  underdogWins['WINNER'] = underdogWins.apply(lambda x: x['AWAY'] if x[0].winner == 'AWAY' else x['HOME'], axis=1)
  
  return underdogWins
  

In [ ]:
underdogWins = getUnderdogWins()
prevPowerMatchupResults = getPrevPowerMatchupResults()
prevPowerMatchupResults

,0,AWAY,AWAY RANK,HOME,HOME RANK,WINNER
0,Box Score(Team(brooklyn park splashers) at Tea...,brooklyn park splashers,2,Team Big tings eh,3,brooklyn park splashers


In [ ]:
getCurrentWeekNum()

3